# Laboratório 3.6 - Notebook do aluno

## Visão geral

Este laboratório é uma continuação dos laboratórios guiados do Módulo 3.

Neste laboratório, você avaliará o modelo treinado em módulos anteriores. Você também calculará métricas com base nos resultados dos dados de teste.


## Introdução ao cenário do negócio

Você trabalha para um provedor de serviços médicos e deseja melhorar a detecção de anormalidades em pacientes ortopédicos. 

Você tem a incumbência de resolver esse problema usando machine learning (ML). Você tem acesso a um conjunto de dados que contém seis funcionalidades biomecânicas (componentes) e um alvo (target) de *normal* (normal) ou *abnormal* (anormal). Você pode usar esse conjunto de dados (dataset) para treinar um modelo de ML para prever se um paciente terá uma anomalia.


## Sobre esse conjunto de dados (dataset)

Esse conjunto de dados biomédicos foi criado pelo Dr. Henrique da Mota durante um período de residência médica no Group of Applied Research in Orthopaedics (GARO) do Centre Médico-Chirurgical de Réadaptation des Massues em Lyon, na França. Os dados foram organizados em duas tarefas de classificação diferentes, mas relacionadas. 

A primeira tarefa consiste em classificar os pacientes como pertencentes a uma das três categorias a seguir: 

- *Normal* (Normal) (100 pacientes)
- *Disk Hernia* (Hérnia de disco) (60 pacientes)
- *Spondylolisthesis* (Espondilolistese) (150 pacientes)

Para a segunda tarefa, as categorias *Disk Hernia* (Hérnia de disco) e *Spondylolisthesis* (Espondilolistese) foram mescladas em uma única categoria, rotulada como *abnormal* (anormal). Portanto, a segunda tarefa consiste em classificar os pacientes como pertencentes a uma das categorias: *Normal* (Normal) (100 pacientes) ou *Abnormal* (Anormal) (210 pacientes).


## Informações dos atributos

Cada paciente é representado no conjunto de dados (dataset) por seis atributos biomecânicos derivados da forma e da orientação da pelve e da coluna lombar (nesta ordem): 

- Incidência pélvica
- Inclinação pélvica
- Ângulo da lordose lombar
- Inclinação sacral
- Raio pélvico
- Grau de espondilolistese

A convenção a seguir é usada para os rótulos de classe (labels): 
- DH (hérnia de disco)
- Espondilolistese (SL)
- Normal (NO) 
- Anormal (AB)

Para obter mais informações sobre esse conjunto de dados (dataset), consulte a [página da Web Conjunto de dados de coluna vertebral](http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuições do conjunto de dados (dataset)

Esse conjunto de dados (dataset) foi obtido de:
Dua, D. e Graff, C. (2019). repositório UCI Machine Learning (http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.


# Configuração do laboratório

Como a solução é dividida em vários laboratórios no módulo, você executará as seguintes células para poder carregar os dados e treinar o modelo a ser implantado.

**Observação:** a configuração pode demorar até cinco minutos para ser concluída.

## Importação dos dados e treinamento do modelo

Ao executar as células a seguir, os dados serão importados e estarão prontos para uso. 

**Observação:** as células a seguir representam as principais etapas dos laboratórios anteriores.


In [ ]:
bucket='c117523a2804662l6883846t1w198478609102-labbucket-ojfl3mti4wra'

In [ ]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

In [ ]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

batch_X = test.iloc[:,1:];

batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),names=['class'])

# Etapa 1: Explorando os resultados

A saída do modelo será uma probabilidade. Você deve, primeiramente, converter essa probabilidade em uma das duas classes, *0* ou *1*. Para fazer isso, você pode criar uma função para executar a conversão. Observe o uso do limite na função.

In [ ]:
def binary_convert(x):
    threshold = 0.3
    if x > threshold:
        return 1
    else:
        return 0

target_predicted_binary = target_predicted['class'].apply(binary_convert)

print(target_predicted_binary.head(5))
test.head(5)

Com base nesses resultados, você pode observar que o modelo inicial pode não ser tão bom. É difícil saber pela comparação de alguns valores.

Em seguida, você gerará algumas métricas para ver o desempenho do modelo.


# Etapa 2: Criando de uma matriz de confusão (confusion matrix)

Uma *matriz de confusão* (confusion matrix) é uma das principais maneiras de medir o desempenho de um modelo de classificação. É uma tabela que mapeia as previsões corretas e incorretas. Depois de calcular uma matriz de confusão (confusion matrix) para o modelo, você poderá gerar várias outras estatísticas. No entanto, você começará criando apenas a matriz de confusão (confusion matrix).

Para criar uma matriz de confusão (confusion matrix), você precisa dos valores algo (target) dos dados de teste *e* do valor predito. 

Obtenha os alvos (targets) do DataFrame de teste.

In [ ]:
test_labels = test.iloc[:,0]
test_labels.head()

Agora, você pode usar a biblioteca *scikit-learn*, que contém uma função para criar uma matriz de confusão (confusion matrix).

In [ ]:

from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(test_labels, target_predicted_binary)
df_confusion = pd.DataFrame(matrix, index=['Nnormal','Abnormal'],columns=['Normal','Abnormal'])

df_confusion

Os resultados variam, mas você deverá ter resultados semelhantes a este exemplo:

_ | Abnormal | Normal (_ | Anormal | Normal)
---------- | ----: | ----:
Normal | 7 | 3
Abnormal | 3 | 18 (Anormal | 3 | 18)



A tabela anterior mostra que o modelo previu corretamente os valores *7 Normal* (7 Normais) e *18 Abnormal* (18 Anormais). No entanto, ele previu incorretamente os valores *3 Normal* (3 Normais) e *3 Abnormal* (3 Anormais). 

Ao usar as bibliotecas Python *seaborn* e *matplotlib*, você pode plotar esses valores em um gráfico para facilitar a leitura.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colormap = sns.color_palette("BrBG", 10)
sns.heatmap(df_confusion, annot=True, cbar=None, cmap=colormap)
plt.title("Confusion Matrix")
plt.tight_layout()
plt.ylabel("True Class")
plt.xlabel("Predicted Class")
plt.show()

**Dica:** se o gráfico não for exibido na primeira vez, tente executar a célula novamente.

Se esses resultados forem suficientemente bons para sua aplicação, isso significa que o modelo pode estar suficientemente bom. No entanto, como há consequências da previsão incorreta dos valores *Normal* (Normais) (ou seja, nenhuma anomalia foi encontrada quando realmente houve uma), o foco deve ser a redução desse resultado.

# Etapa 3: Cálculo dos dados estatísticos de desempenho

Se você quiser comparar esse modelo com o próximo modelo que criar, precisará de algumas métricas que possam ser registradas. Para um problema de classificação binária, os dados da matriz de confusão (confusion matrix) podem ser usados para calcular várias métricas.

Para começar, extraia os valores das células da matriz de confusão (confusion matrix) em variáveis.

In [ ]:

from sklearn.metrics import roc_auc_score, roc_curve, auc

TN, FP, FN, TP = confusion_matrix(test_labels, target_predicted_binary).ravel()

print(f"True Negative (TN) : {TN}")
print(f"False Positive (FP): {FP}")
print(f"False Negative (FN): {FN}")
print(f"True Positive (TP) : {TP}")

Agora, você pode calcular alguns dados estatísticos.


### Sensibilidade

A *Sensibilidade* (sensitivity) também é conhecida como *taxa de acertos* *hit rate* , *recall* ou *taxa de verdadeiros positivos (TPR)*. Ela mede a proporção dos verdadeiros positivos que são identificados corretamente.

Neste exemplo, a sensibilidade é *a probabilidade de detecção de uma anormalidade em pacientes com uma anormalidade*.

In [ ]:
# Sensitivity, hit rate, recall, or true positive rate
Sensitivity  = float(TP)/(TP+FN)*100
print(f"Sensitivity or TPR: {Sensitivity}%")  
print(f"There is a {Sensitivity}% chance of detecting patients with an abnormality have an abnormality")

**Pergunta:** A sensibilidade é suficientemente boa para este cenário?


### Especificidade

O próximo dado estatístico é a *especificidade* (specificity), que também é conhecida como *verdadeiro negativo*. Ela mede a proporção dos verdadeiros negativos que são identificados corretamente.

Neste exemplo, a especificidade é *a probabilidade de detecção de normalidade em pacientes normais*.

In [ ]:
# Specificity or true negative rate
Specificity  = float(TN)/(TN+FP)*100
print(f"Specificity or TNR: {Specificity}%") 
print(f"There is a {Specificity}% chance of detecting normal patients are normal.")


**Pergunta:** Essa especificidade é muito baixa, exatamente correta ou muito alta? Que valor você gostaria de ver aqui, considerando o cenário?



### Valores preditivos positivos e negativos

A *precisão*, ou *valor preditivo positivo*, é a proporção de resultados positivos.

Neste exemplo, o valor preditivo positivo é *a probabilidade de que os indivíduos com teste de triagem positivo tenham, realmente, uma anormalidade*.

In [ ]:
# Precision or positive predictive value
Precision = float(TP)/(TP+FP)*100
print(f"Precision: {Precision}%")  
print(f"You have an abnormality, and the probablity that is correct is {Precision}%")

O *valor preditivo negativo* é a proporção de resultados negativos.

Neste exemplo, o valor preditivo negativo é *a probabilidade de que os indivíduos com teste de triagem negativo tenham, realmente, uma anormalidade*.

In [ ]:
# Negative predictive value
NPV = float(TN)/(TN+FN)*100
print(f"Negative Predictive Value: {NPV}%") 
print(f"You don't have an abnormality, but there is a {NPV}% chance that is incorrect" )

Pense no impacto desses valores. Se você fosse um paciente, até que ponto você estaria preocupado se o teste para uma anormalidade fosse positivo? Por outro lado, até que ponto você estaria tranquilo se tivesse testado negativo?


### Taxa de falsos positivos (false positive rate)

A *taxa de falsos positivos (FPR)* (false positive rate) é a probabilidade de um alarme falso ocorrer ou *um resultado positivo ser fornecido quando um valor verdadeiro for negativo*. 

In [ ]:
# Fall out or false positive rate
FPR = float(FP)/(FP+TN)*100
print( f"False Positive Rate: {FPR}%") 
print( f"There is a {FPR}% chance that this positive result is incorrect.")

### Taxa de falsos negativos

A *taxa de falsos negativos* (false negative rate) - ou *taxa de erro* (miss rate) - é *a probabilidade de que um verdadeiro positivo não seja detectado pelo teste*.

In [ ]:
# False negative rate
FNR = float(FN)/(TP+FN)*100
print(f"False Negative Rate: {FNR}%") 
print(f"There is a {FNR}% chance that this negative result is incorrect.")

### Taxa de descobertas falsas (False discovery rate)

Neste exemplo, a *taxa de descoberta falsa* é *a probabilidade de se prever uma anormalidade quando o paciente não possuir uma*.

In [ ]:
# False discovery rate
FDR = float(FP)/(TP+FP)*100
print(f"False Discovery Rate: {FDR}%" )
print(f"You have an abnormality, but there is a {FDR}% chance this is incorrect.")

### Precisão geral

Qual a precisão do seu modelo?

In [ ]:
# Overall accuracy
ACC = float(TP+TN)/(TP+FP+FN+TN)*100
print(f"Accuracy: {ACC}%") 

Em resumo, você calculou as seguintes métricas do modelo:

In [ ]:
print(f"Sensitivity or TPR: {Sensitivity}%")    
print(f"Specificity or TNR: {Specificity}%") 
print(f"Precision: {Precision}%")   
print(f"Negative Predictive Value: {NPV}%")  
print( f"False Positive Rate: {FPR}%") 
print(f"False Negative Rate: {FNR}%")  
print(f"False Discovery Rate: {FDR}%" )
print(f"Accuracy: {ACC}%") 

**Tarefa desafio:** Registre os valores anteriores e, em seguida, volte para a etapa 1 e altere o valor usado para limite (threshold). Os valores que você deverá tentar são *0,25* e *0,75*. 

Esses valores de limite (threshold) fazem alguma diferença?

# Etapa 4: Cálculo do AUC-ROC

A biblioteca scikit-learn tem funções que podem ajudar você a calcular a *área sob a curva característica de operação do receptor (AUC-ROC)*.

- A ROC é uma curva de probabilidade.
- A AUC informa o quanto o modelo pode fazer a distinção entre classes. 

A AUC pode ser calculada. Como você verá no próximo laboratório, ela pode ser usada para medir o desempenho do modelo. 

Neste exemplo, quanto maior for a AUC, melhor o modelo fará distinção entre pacientes anormais e normais.

Dependendo do valor definido para o limite (threshold), a AUC pode mudar. Você pode plotar a AUC usando a probabilidade em vez da classe convertida.


In [ ]:
test_labels = test.iloc[:,0];
print("Validation AUC", roc_auc_score(test_labels, target_predicted) )

Normalmente, a curva ROC é representada com a TPR em relação à FPR, em que a TPR está no eixo y e a FPR está no eixo x.

O scikit-learn tem a função **roc_curve** para ajudar a gerar esses valores para plotagem.

In [ ]:
fpr, tpr, thresholds = roc_curve(test_labels, target_predicted)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
 
# create the axis of thresholds (scores)
ax2 = plt.gca().twinx()
ax2.plot(fpr, thresholds, markeredgecolor='r',linestyle='dashed', color='r')
ax2.set_ylabel('Threshold',color='r')
ax2.set_ylim([thresholds[-1],thresholds[0]])
ax2.set_xlim([fpr[0],fpr[-1]])

print(plt.figure())

**Tarefa de desafio:** Atualize o código anterior para usar *target_predicted_binary* em vez de *target_predicted*. Como isso altera o gráfico? Qual é o mais útil?

# Parabéns!

Você concluiu este laboratório e agora pode encerrá-lo seguindo as instruções do guia do laboratório.